In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

In [ ]:
base_url = 'https://developers.themoviedb.org/3/discover/movie-discover'